In [1]:
import math
import numpy as np
import sys
from itertools import product

# Size of matrices
M = 16
K = 64
N = 512

M = 256
K = 64
N = 512

# Size of buffers
A_buffer_size = 4096
B_buffer_size = 4096
C_buffer_size = 4096


In [2]:
tile_M = 16
tile_N = 16
tile_K = 16

In [3]:
def create_matrices(M, N, K):
    """
    Creates matrices A and B.
    M: integer representing the number of rows of A
    N: integer representing the number of columns of B
    K: integer representing the number of columns of A and rows of B
    returns: two NumPy arrays representing the matrices A and B
    """
    A = np.zeros((M, K))
    B = np.zeros((K, N))

    for i in range(M):
        A[i, :] = i

    for j in range(N):
        B[:, j] = j

    return A, B


In [26]:
def a_stationary_matmul(A, B, tile_M, tile_K, tile_N):
    """
    Performs A-stationary matrix multiplication on matrices A and B and returns the result as an int32 NumPy array.
    A: a NumPy array representing the matrix A with size MxK
    B: a NumPy array representing the matrix B with size KxN
    tile_M: integer representing the size of the tiles in the first dimension (M)
    tile_K: integer representing the size of the tiles in the second dimension (K)
    tile_N: integer representing the size of the tiles in the third dimension (N)
    """
    M, K = A.shape
    K, N = B.shape
    C = np.zeros((M, N))
    C_tile = np.zeros((tile_M, tile_N))

    num_accesses_A = 0
    num_accesses_B = 0
    num_accesses_C = 0

    for i in range(0, M, tile_M):
        for k in range(0, K, tile_K):
            A_tile = A[i:i+tile_M, k:k+tile_K]
            num_accesses_A += A_tile.size
            for j in range(0, N, tile_N):
                C_tile.fill(0)  # Reset C_tile to all zeros for each new tile of C
                B_tile = B[k:k+tile_K, j:j+tile_N]
                num_accesses_B += B_tile.size
                C_tile += np.matmul(A_tile, B_tile)
                num_accesses_C += C_tile.size
                C[i:i+tile_M, j:j+tile_N] += C_tile

    return C, num_accesses_A, num_accesses_B, num_accesses_C

In [5]:
def b_stationary_matmul(A, B, tile_M, tile_K, tile_N):
    """
    Performs B-stationary matrix multiplication on matrices A and B and returns the result as an int32 NumPy array.
    A: a NumPy array representing the matrix A with size MxK
    B: a NumPy array representing the matrix B with size KxN
    tile_M: integer representing the size of the tiles in the first dimension (M)
    tile_K: integer representing the size of the tiles in the second dimension (K)
    tile_N: integer representing the size of the tiles in the third dimension (N)
    """
    M, K = A.shape
    K, N = B.shape
    C = np.zeros((M, N))
    C_tile = np.zeros((tile_M, tile_N))
    
    num_accesses_A = 0
    num_accesses_B = 0
    num_accesses_C = 0

    for k in range(0, K, tile_K):
        for j in range(0, N, tile_N):
            B_tile = B[k:k+tile_K, j:j+tile_N]
            num_accesses_B += B_tile.size
            for i in range(0, M, tile_M):
                C_tile.fill(0)  # Reset C_tile to all zeros for each new tile of C
                A_tile = A[i:i+tile_M, k:k+tile_K]
                num_accesses_A += A_tile.size
                C_tile += np.matmul(A_tile, B_tile)
                num_accesses_C += C_tile.size
                C[i:i+tile_M, j:j+tile_N] += C_tile

    return C, num_accesses_A, num_accesses_B, num_accesses_C
    

In [6]:
def c_stationary_matmul(A, B, tile_M, tile_K, tile_N):
    """
    Performs C-stationary matrix multiplication on matrices A and B and returns the result as an int32 NumPy array.
    A: a NumPy array representing the matrix A with size MxK
    B: a NumPy array representing the matrix B with size KxN
    tile_M: integer representing the size of the tiles in the first dimension (M)
    tile_K: integer representing the size of the tiles in the second dimension (K)
    tile_N: integer representing the size of the tiles in the third dimension (N)
    """
    M, K = A.shape
    K, N = B.shape
    C = np.zeros((M, N))
    C_tile = np.zeros((tile_M, tile_N))

    num_accesses_A = 0
    num_accesses_B = 0
    num_accesses_C = 0

    for i in range(0, M, tile_M):
        for j in range(0, N, tile_N):
            C_tile.fill(0)  # Reset C_tile to all zeros for each new tile of C
            for k in range(0, K, tile_K):
                A_tile = A[i:i+tile_M, k:k+tile_K]
                num_accesses_A += A_tile.size
                B_tile = B[k:k+tile_K, j:j+tile_N]
                num_accesses_B += B_tile.size
                C_tile += np.matmul(A_tile, B_tile)
            num_accesses_C += C_tile.size
            C[i:i+tile_M, j:j+tile_N] += C_tile

    return C, num_accesses_A, num_accesses_B, num_accesses_C

In [7]:
def a_stationary_num_accesses(M, N, K, tile_M, tile_K, tile_N):
    num_tiles_A = math.ceil(M / tile_M) * math.ceil(K / tile_K)
    num_tiles_B = math.ceil(K / tile_K) * math.ceil(N / tile_N) * math.ceil(M / tile_M)
    num_tiles_C = math.ceil(M / tile_M) * math.ceil(N / tile_N) * math.ceil(K / tile_K)

    num_accesses_A = num_tiles_A * tile_M * tile_K
    num_accesses_B = num_tiles_B * tile_K * tile_N
    num_accesses_C = num_tiles_C * tile_M * tile_N

    return num_accesses_A, num_accesses_B, num_accesses_C

In [8]:
def b_stationary_num_accesses(M, N, K, tile_M, tile_K, tile_N):
    num_tiles_A = math.ceil(K / tile_K) * math.ceil(M / tile_M) * math.ceil(N / tile_N)
    num_tiles_B = math.ceil(K / tile_K) * math.ceil(N / tile_N)
    num_tiles_C = math.ceil(M / tile_M) * math.ceil(N / tile_N) * math.ceil(K / tile_K)

    num_accesses_A = num_tiles_A * tile_M * tile_K
    num_accesses_B = num_tiles_B * tile_K * tile_N
    num_accesses_C = num_tiles_C * tile_M * tile_N

    return num_accesses_A, num_accesses_B, num_accesses_C

In [9]:
def c_stationary_num_accesses(M, N, K, tile_M, tile_K, tile_N):
    num_tiles_A = math.ceil(K / tile_K) * math.ceil(M / tile_M) * math.ceil(N / tile_N)
    num_tiles_B = math.ceil(K / tile_K) * math.ceil(N / tile_N) * math.ceil(M / tile_M)
    num_tiles_C = math.ceil(M / tile_M) * math.ceil(N / tile_N)

    num_accesses_A = num_tiles_A * tile_M * tile_K
    num_accesses_B = num_tiles_B * tile_K * tile_N
    num_accesses_C = num_tiles_C * tile_M * tile_N
    return num_accesses_A, num_accesses_B, num_accesses_C

In [10]:
def print_best(outputs):
    least = sys.maxsize
    min_index = 0
    for i,data in enumerate(outputs):
        if data[7] < least:
            least = data[7]
            min_index = i

    bests = []
    for i,data in enumerate(outputs):
        if data[7] == least:
            bests.append(data)
    return bests

In [22]:
A,B = create_matrices(M,N,K)
(output,a,b,c) = a_stationary_matmul(A,B,tile_M,tile_K,tile_N)
print("A accesses: ", a)
print("B accesses: ", b)
print("C accesses: ", c)

(a,b,c) = a_stationary_num_accesses(M,N,K,tile_M,tile_K,tile_N)
print("A accesses: ", a)
print("B accesses: ", b)
print("C accesses: ", c)

A accesses:  131072
B accesses:  524288
C accesses:  524288
A accesses:  131072
B accesses:  524288
C accesses:  524288


In [29]:
M = 512
N = 64
K = 256
tile_M = 64
tile_N = 16
tile_K = 4

A,B = create_matrices(M,N,K)
(output,a,b,c) = a_stationary_matmul(A,B,tile_M,tile_K,tile_N)
print("A accesses: ", a)
print("B accesses: ", b)
print("C accesses: ", c)

(a,b,c) = a_stationary_num_accesses(M,N,K,tile_M,tile_K,tile_N)
print("A accesses: ", a)
print("B accesses: ", b)
print("C accesses: ", c)

A accesses:  512
B accesses:  2048
C accesses:  2048
A accesses:  131072
B accesses:  131072
C accesses:  2097152


In [30]:
M = 512
N = 64
K = 256
tile_M = 16
tile_N = 64
tile_K = 4

A,B = create_matrices(M,N,K)
(output,a,b,c) = a_stationary_matmul(A,B,tile_M,tile_K,tile_N)
print("A accesses: ", a)
print("B accesses: ", b)
print("C accesses: ", c)

(a,b,c) = a_stationary_num_accesses(M,N,K,tile_M,tile_K,tile_N)
print("A accesses: ", a)
print("B accesses: ", b)
print("C accesses: ", c)

A accesses:  2048
B accesses:  2048
C accesses:  2048
A accesses:  131072
B accesses:  524288
C accesses:  2097152


In [12]:
tile_sizes = [16, 32]
max_tile_size = 64

num_runs = len(tile_sizes) ** 3
outputs2 = []

count = 0
for tile_M, tile_K, tile_N in product(tile_sizes, repeat=3):
    if tile_N * tile_K <= max_tile_size ** 2 and tile_M * tile_K <= max_tile_size ** 2 and tile_N * tile_M <= max_tile_size ** 2:

        num_accesses_A, num_accesses_B, num_accesses_C =  a_stationary_num_accesses(M,N,K,tile_M,tile_K,tile_N)
        outputs2.append(["A",tile_M, tile_K, tile_N,num_accesses_A, num_accesses_B, num_accesses_C])

        num_accesses_A, num_accesses_B, num_accesses_C =  b_stationary_num_accesses(M,N,K,tile_M,tile_K,tile_N)
        outputs2.append(["B",tile_M, tile_K, tile_N,num_accesses_A, num_accesses_B, num_accesses_C])

        num_accesses_A, num_accesses_B, num_accesses_C =  c_stationary_num_accesses(M,N,K,tile_M,tile_K,tile_N)
        outputs2.append(["C",tile_M, tile_K, tile_N,num_accesses_A, num_accesses_B, num_accesses_C])

        count += 1




# for i,j in zip(outputs,outputs2):
#     print(j)

In [13]:
M = 2048
K = 512
N = 128

# Size of buffers
tile_sizes = [i  for i in range(16,257,16)]

def get_access_count(M,N,K,tile_sizes):
    A_buffer_size = 4096
    B_buffer_size = 4096
    C_buffer_size = 4096
    outputs = []
    count = 0
    print("For problem size: ", M, N, K, " and buffer sizes: ", A_buffer_size, B_buffer_size, C_buffer_size)
    for tile_M, tile_K, tile_N in product(tile_sizes, repeat=3):
        if tile_N * tile_K <= A_buffer_size and tile_M * tile_K <= B_buffer_size and tile_N * tile_M <= C_buffer_size:
            num_accesses_A, num_accesses_B, num_accesses_C =  a_stationary_num_accesses(M,N,K,tile_M,tile_K,tile_N)
            sum_accesses = num_accesses_A + num_accesses_B + num_accesses_C
            outputs.append(["A",tile_M, tile_K, tile_N,num_accesses_A, num_accesses_B, num_accesses_C,sum_accesses])
            num_accesses_A, num_accesses_B, num_accesses_C =  b_stationary_num_accesses(M,N,K,tile_M,tile_K,tile_N)
            sum_accesses = num_accesses_A + num_accesses_B + num_accesses_C
            outputs.append(["B",tile_M, tile_K, tile_N,num_accesses_A, num_accesses_B, num_accesses_C,sum_accesses])
            num_accesses_A, num_accesses_B, num_accesses_C =  c_stationary_num_accesses(M,N,K,tile_M,tile_K,tile_N)
            sum_accesses = num_accesses_A + num_accesses_B + num_accesses_C
            outputs.append(["C",tile_M, tile_K, tile_N,num_accesses_A, num_accesses_B, num_accesses_C,sum_accesses])
            count += 1

    b=print_best(outputs)
    for i in b:
        print(i)

In [14]:
dims = [[16,512,2048],[64,512,512]]
tile_sizes = [i  for i in range(16,257,16)]

for dim in dims:
    M = dim[0]
    N = dim[1]
    K = dim[2]
    get_access_count(M,N,K,tile_sizes)


For problem size:  16 512 2048  and buffer sizes:  4096 4096 4096
['C', 16, 16, 256, 65536, 1048576, 8192, 1122304]
For problem size:  64 512 512  and buffer sizes:  4096 4096 4096
['C', 64, 16, 64, 262144, 262144, 32768, 557056]
['C', 64, 32, 64, 262144, 262144, 32768, 557056]
['A', 64, 64, 16, 32768, 262144, 262144, 557056]
['A', 64, 64, 32, 32768, 262144, 262144, 557056]
['A', 64, 64, 64, 32768, 262144, 262144, 557056]
['C', 64, 64, 64, 262144, 262144, 32768, 557056]
